In [20]:
# Link dataset
# https://drive.google.com/file/d/1UW38R58WU9KLcypftUMXGa2h504wdC86/view?usp=sharing
# Link map by book_id to book title
# https://drive.google.com/file/d/1IguDcCBQ2NMl20By9VEjJKg2HmelGqks/view?usp=sharing

# Lưu ý:
# Viết code rõ ràng, nhớ có comment, hoàn thiện mới được push lên nhánh mast

In [54]:
import numpy as np
import pandas as pd
from sklearn.feature_extraction import DictVectorizer
from sklearn.metrics.pairwise import cosine_similarity

In [55]:
df = pd.read_csv('./goodreads_reviews_comics_graphic.csv')

In [56]:
map_title = pd.read_csv('./goodreads_books_comics_graphic.csv')[['book_id', 'title', 'url']]

In [24]:
map_title.head()

,book_id,title,url
0,25742454,The Switchblade Mamma,https://www.goodreads.com/book/show/25742454-t...
1,30128855,Cruelle,https://www.goodreads.com/book/show/30128855-c...
2,13571772,Captain America: Winter Soldier (The Ultimate ...,https://www.goodreads.com/book/show/13571772-c...
3,35452242,Bounty Hunter 4/3: My Life in Combat from Mari...,https://www.goodreads.com/book/show/35452242-b...
4,707611,"Superman Archives, Vol. 2",https://www.goodreads.com/book/show/707611.Sup...


In [57]:
map_title.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 89411 entries, 0 to 89410
Data columns (total 3 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   book_id  89411 non-null  int64 
 1   title    89411 non-null  object
 2   url      89411 non-null  object
dtypes: int64(1), object(2)
memory usage: 2.0+ MB


In [58]:
df = df.iloc[0: 500000, :]

In [59]:
map_title.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 89411 entries, 0 to 89410
Data columns (total 3 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   book_id  89411 non-null  int64 
 1   title    89411 non-null  object
 2   url      89411 non-null  object
dtypes: int64(1), object(2)
memory usage: 2.0+ MB


In [60]:
len(df['book_id'].unique())

86262

In [61]:
df.head()

,user_id,book_id,review_id,rating,review_text,date_added,date_updated,read_at,started_at,n_votes,n_comments
0,dc3763cdb9b2cae805882878eebb6a32,18471619,66b2ba840f9bd36d6d27f46136fe4772,3,Sherlock Holmes and the Vampires of London \r\...,Thu Dec 05 10:44:25 -0800 2013,Thu Dec 05 10:45:15 -0800 2013,Tue Nov 05 00:00:00 -0800 2013,NaN,0,0
1,bafc2d50014200cda7cb2b6acd60cd73,6315584,72f1229aba5a88f9e72f0dcdc007dd22,4,"I've never really liked Spider-Man. I am, howe...",Wed Aug 10 06:06:48 -0700 2016,Fri Aug 12 08:49:54 -0700 2016,Fri Aug 12 08:49:54 -0700 2016,Wed Aug 10 00:00:00 -0700 2016,0,0
2,bafc2d50014200cda7cb2b6acd60cd73,29847729,a75309355f8662caaa5e2c92ab693d3f,4,"A very quick introduction, this is coming out ...",Thu Apr 21 07:44:00 -0700 2016,Thu Apr 21 07:59:28 -0700 2016,Thu Apr 21 07:59:28 -0700 2016,Thu Apr 21 00:00:00 -0700 2016,0,0
3,bafc2d50014200cda7cb2b6acd60cd73,18454118,c3cc5a3e1d6b6c9cf1c044f306c8e752,5,I've been waiting so long for this. I first st...,Mon Mar 03 17:45:56 -0800 2014,Mon Mar 03 17:54:11 -0800 2014,Sat Mar 01 00:00:00 -0800 2014,Sat Mar 01 00:00:00 -0800 2014,1,0
4,bafc2d50014200cda7cb2b6acd60cd73,2239435,cc444be37ab0a42bfb4dd818cb5edd10,4,The only thing more entertaining than this boo...,Wed Apr 03 12:37:48 -0700 2013,Wed Apr 03 13:03:36 -0700 2013,Wed Apr 03 13:03:36 -0700 2013,NaN,0,0


In [62]:
# Chỉ lấy user_id, book_id, rating, n_votes để xử lí
# rating cao cùng với n_votes(số lượng vote cho comment) càng cao thì quyển sách đó hay, đã được nhiều người đánh giá
data = df[['user_id', 'book_id', 'rating', 'n_votes']]

In [63]:
data.head()
# Bắt đầu xử lí và áp dụng giải thuật từ đây

,user_id,book_id,rating,n_votes
0,dc3763cdb9b2cae805882878eebb6a32,18471619,3,0
1,bafc2d50014200cda7cb2b6acd60cd73,6315584,4,0
2,bafc2d50014200cda7cb2b6acd60cd73,29847729,4,0
3,bafc2d50014200cda7cb2b6acd60cd73,18454118,5,1
4,bafc2d50014200cda7cb2b6acd60cd73,2239435,4,0


In [64]:
data.describe()[['rating', 'n_votes']]

,rating,n_votes
count,500000.000000,500000.000000
mean,3.773792,0.982644
std,1.152686,5.074714
min,0.000000,-2.000000
25%,3.000000,0.000000
50%,4.000000,0.000000
75%,5.000000,1.000000
max,5.000000,753.000000


In [65]:
listOfDictonaries=[]
indexMap = {}
reverseIndexMap = {}
ptr=0
testdf = data
# Group lại theo book_id
testdf=testdf.groupby('book_id')[['user_id', 'rating']]

In [66]:
# Duyệt qua tất cả book_id
for groupKey in testdf.groups.keys():
    tempDict = {}
    groupDF = testdf.get_group(groupKey)
    # Duyệt qua tất cả dữ liệu rating, n_vote đối với tất cả book_id
    for i in range(0, len(groupDF)):
        # Gán user_id và rating của user đó vào một dict_list
        tempDict[groupDF.iloc[i, 0]] = groupDF.iloc[i, 1]
    indexMap[ptr]=groupKey
    reverseIndexMap[groupKey] = ptr
    ptr+=1
    listOfDictonaries.append(tempDict)

In [35]:
print(list(indexMap.items())[:4])
print(list(reverseIndexMap.items())[:4])
print(len(indexMap))

[(0, 96358), (1, 188172), (2, 228356), (3, 384897)]
[(96358, 0), (188172, 1), (228356, 2), (384897, 3)]
20


In [36]:
#Skip itttttttttt
temp={}
group = testdf.get_group(2239435)
for i in range(0, len(group)):
    temp[group.iloc[i, 0]] = group.iloc[i, 1]
temp
# {'user_id': raiting}

KeyError: 2239435

In [402]:
print(listOfDictonaries)

[{'af157d0205b8a901dee6d4a2aed7e6ad': 5}, {'0d8d07544717e84149df654caae803d0': 5}, {'92b4a378f7007946263f655b23b6f639': 4}, {'2f6af21d14c83a5df6cdcef5e6af0b3e': 4}, {'faa322d2624b0e7eb3064e39dac4af9c': 1}, {'faa322d2624b0e7eb3064e39dac4af9c': 2}, {'faa322d2624b0e7eb3064e39dac4af9c': 4}, {'0d8d07544717e84149df654caae803d0': 5}, {'478f3accbbb16d817fd90732215b6391': 3}, {'d52ba256a0ff3980e123b4e0a5d52a34': 5}, {'4672eb229c808b792b8ea95f01f19784': 2}, {'617ccec66dac2d1029600ed3d706e8ed': 4}, {'5e09e38e178edf7b524913c729854741': 3}, {'bda99715e8f6865c236926a0979b72a5': 0}, {'869f7931c730798de6310de9445453e7': 4}, {'559d843b319087e12f48282e386e401f': 4}, {'617ccec66dac2d1029600ed3d706e8ed': 5}, {'2f6af21d14c83a5df6cdcef5e6af0b3e': 5}, {'74923eed26118d852196f9aa18070272': 4}, {'74923eed26118d852196f9aa18070272': 5}]


In [73]:
dictVectorizer = DictVectorizer(dtype = np.uint64, sparse=True)
vector = dictVectorizer.fit_transform(listOfDictonaries)

In [72]:
pd.DataFrame(vector.todense().T)

MemoryError: Unable to allocate 32.4 GiB for an array with shape (86262, 50475) and data type uint64

In [70]:
pairwiseSimilarity = cosine_similarity(vector)

MemoryError: Unable to allocate 55.4 GiB for an array with shape (86262, 86262) and data type float64

In [71]:
pd.DataFrame(pairwiseSimilarity)

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19
0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
4,0.0,0.0,0.0,0.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5,0.0,0.0,0.0,0.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
6,0.0,0.0,0.0,0.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
7,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
8,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
9,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [49]:
def printBookDetails(bookID):
    print("Book-ID:",bookID)
    print("Title:", map_title[map_title['book_id']==bookID]['title'].values[0])
    print("URL:",map_title[map_title['book_id']==bookID]['url'].values[0])
    print("=================++++++++++++++=========================\n\n")

def getTopRecommandations(bookID):
    row = reverseIndexMap[bookID]
    print("------INPUT BOOK--------")
    printBookDetails(bookID)
    print("-------RECOMMENDATIONS----------")
    similarBookIDs = [printBookDetails(indexMap[i]) for i in np.argsort(pairwiseSimilarity[row])[-7:-2][::-1]]
    

In [335]:
getTopRecommandations(32332710)

------INPUT BOOK--------
Book-ID: 32332710
Title: Silly Sea Stories (SpongeBob Comics #1)
URL: https://www.goodreads.com/book/show/32332710-silly-sea-stories
=================++++++++++++++=========================


-------RECOMMENDATIONS----------
Book-ID: 1003034
Title: Kilala Princess, Vol. 02
URL: https://www.goodreads.com/book/show/1003034.Kilala_Princess_Vol_02
=================++++++++++++++=========================


Book-ID: 32332713
Title: Aquatic Adventurers, Unite! (SpongeBob Comics #2)
URL: https://www.goodreads.com/book/show/32332713-aquatic-adventurers-unite
=================++++++++++++++=========================


Book-ID: 32332710
Title: Silly Sea Stories (SpongeBob Comics #1)
URL: https://www.goodreads.com/book/show/32332710-silly-sea-stories
=================++++++++++++++=========================


Book-ID: 6186053
Title: The Shadow Door
URL: https://www.goodreads.com/book/show/6186053-the-shadow-door
=================++++++++++++++=========================


Book

In [337]:
df = pd.DataFrame({'Animal': ['Falcon', 'Falcon',

                              'Parrot', 'Parrot'],

                   'Max Speed': [380., 370., 24., 26.]})

In [338]:
a = df.groupby(['Animal']).mean()

In [339]:
a.head()

,Max Speed
Animal,
Falcon,375.0
Parrot,25.0
